In [1]:
import numpy as np
import tensorflow as tf

In [65]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float32),npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float32),npz['targets'].astype(np.int32)

In [93]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_inputs = scaler.fit_transform(train_inputs)
validation_inputs = scaler.transform(validation_inputs)
test_inputs = scaler.transform(test_inputs)

# Define model with Functional API
inputs_size = train_inputs.shape[1]  # Dynamically get input size
inputs = tf.keras.Input(shape=(inputs_size,))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Residual block
residual = x
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.add([x, residual])  # Skip connection

outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [94]:
batch_size = 32

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=5, restore_best_weights=True
)

history = model.fit(
    train_inputs, train_targets,
    batch_size=64,
    epochs=200,
    validation_data=(validation_inputs, validation_targets),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5805 - loss: 1.0205 - val_accuracy: 0.6853 - val_loss: 0.6248
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6697 - loss: 0.7284 - val_accuracy: 0.6384 - val_loss: 0.6449
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6577 - loss: 0.7009 - val_accuracy: 0.6696 - val_loss: 0.6047
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6739 - loss: 0.6589 - val_accuracy: 0.6674 - val_loss: 0.6037
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6624 - loss: 0.6554 - val_accuracy: 0.6920 - val_loss: 0.5885
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6777 - loss: 0.6178 - val_accuracy: 0.7188 - val_loss: 0.5632
Epoch 7/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6783 - loss: 0.6080 - val_accuracy: 0.7009 - val_loss: 0.5780
Epoch 8/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6850 - loss: 0.5963 - val_accuracy: 0.7254 

In [95]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7236 - loss: 0.5461  

Test loss: 0.54. Test accuracy: 73.21%
